In [ ]:
import sys

sys.path.insert(1, "../")
import glob
from dem_comparison.plots import plot_metrics
from osgeo import gdal
from pathlib import Path
import shutil
import glob

In [ ]:
is_error = True
polar = True
pkls = glob.glob(f"../Antarctica_Dem_Comparison/dem_diff_metrics/**.pkl")
plot_metrics(pkls, is_error, polar, "plot.html")

In [ ]:
overviews_path = Path("../Antarctica_Dem_Comparison/overviews")
overviews_path.mkdir(parents=True, exist_ok=True)
mosaic_files = [
    Path(f) for f in glob.glob("../Antarctica_Dem_Comparison/mosaics/*.tif")
]
for i, file in enumerate(mosaic_files):
    print(f"Processing {i} of {len(mosaic_files)}.")
    raster = gdal.Open(str(file), 0)
    gdal.SetConfigOption("COMPRESS_OVERVIEW LERC", "DEFLATE")
    raster.BuildOverviews("BILINEAR", [4, 8], gdal.TermProgress_nocb)
    shutil.move(
        f"{str(file).replace(".tif", ".tif.ovr")}",
        overviews_path / f"{file.name.replace(".tif", ".tif.ovr")}",
    )